# Density 
Script to calculate the column density and volume densities

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u

In [6]:
file_msd = './data/msd_reprojected.fits'
hdu_msd = fits.open(file_msd)
data_msd = hdu_msd[0].data
header_msd = hdu_msd[0].header

In [7]:
distance = 2.9e3 *u.kpc